In [15]:
# Import necessary libraries
from collections import defaultdict, Counter

# FP-Growth Implementation
class TreeNode:
    def __init__(self, name, count, parent):
        self.name = name
        self.count = count
        self.node_link = None
        self.parent = parent
        self.children = {}

    def increment(self, count):
        self.count += count

def update_header(node, target):
    while node.node_link is not None:
        node = node.node_link
    node.node_link = target

def update_tree(items, root, header, count):
    if items[0] in root.children:
        root.children[items[0]].increment(count)
    else:
        root.children[items[0]] = TreeNode(items[0], count, root)
        if header[items[0]][1] is None:
            header[items[0]][1] = root.children[items[0]]
        else:
            update_header(header[items[0]][1], root.children[items[0]])
    if len(items) > 1:
        update_tree(items[1:], root.children[items[0]], header, count)

def create_tree(transactions, min_support):
    header = {}
    for trans in transactions:
        for item in trans:
            header[item] = header.get(item, 0) + transactions[trans]
    for item in list(header):
        if header[item] < min_support:
            del(header[item])
    freq_items = set(header.keys())
    if len(freq_items) == 0:
        return None, None
    for item in header:
        header[item] = [header[item], None]
    root = TreeNode('Null', 1, None)
    for trans, count in transactions.items():
        local_items = {}
        for item in trans:
            if item in freq_items:
                local_items[item] = header[item][0]
        if len(local_items) > 0:
            ordered_items = [v[0] for v in sorted(local_items.items(), key=lambda p: p[1], reverse=True)]
            update_tree(ordered_items, root, header, count)
    return root, header

def ascend_tree(node, path):
    if node.parent is not None:
        path.append(node.name)
        ascend_tree(node.parent, path)

def find_prefix_path(base_item, node):
    patterns = {}
    while node is not None:
        path = []
        ascend_tree(node, path)
        if len(path) > 1:
            patterns[tuple(path[1:])] = node.count
        node = node.node_link
    return patterns

def mine_tree(root, header, min_support, prefix, freq_items):
    sorted_items = [item[0] for item in sorted(header.items(), key=lambda p: p[1][0], reverse=True)]
    for base_item in sorted_items:
        new_prefix = prefix.copy()
        new_prefix.add(base_item)
        freq_items.append((new_prefix, header[base_item][0]))
        cond_patterns = find_prefix_path(base_item, header[base_item][1])
        cond_tree, cond_header = create_tree(cond_patterns, min_support)
        if cond_header is not None:
            mine_tree(cond_tree, cond_header, min_support, new_prefix, freq_items)

# Apriori and Association Rule Implementation
def generate_rules(freq_items, min_conf):
    rules = []
    for itemset, _ in freq_items:
        if len(itemset) > 1:
            for consequence in itemset:
                antecedent = itemset - {consequence}
                if antecedent:
                    support_antecedent = get_support(antecedent)
                    support_itemset = get_support(itemset)
                    confidence = support_itemset / support_antecedent
                    if confidence >= min_conf:
                        rules.append((antecedent, consequence, confidence))
    return rules

def get_support(itemset):
    support = 0
    for trans in transactions:
        if itemset.issubset(trans):
            support += 1
    return support / len(transactions)

# Recommendation System Implementation
def recommend(rules, transaction):
    rec_counts = Counter()
    for antecedent, consequence, confidence in rules:
        if antecedent.issubset(transaction) and consequence not in transaction:
            rec_counts[consequence] += 1
    sorted_recs = sorted(rec_counts, key=rec_counts.get, reverse=True)
    return sorted_recs

# Sample Data
transactions = [['668620ad2aab37a0b98a81b8'], ['668621eb69f8b24189420752', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6'], ['6683c66d50c542b909c4f48a', '6680e29817893c388c6e089f', '6681034c272da5e070160b76'], ['668621eb69f8b24189420752', '6686213a7e00841dd3b1e343', '6680d9f7a0b0687efa469826', '6681449b175ae36ae74b5b27', '6683c66d50c542b909c4f48a', '6686213a7e00841dd3b1e343'], ['668620ad2aab37a0b98a81b8', '668620ad2aab37a0b98a81b8'], ['668620ad2aab37a0b98a81b8', '668620ad2aab37a0b98a81b8', '668620ad2aab37a0b98a81b8', '668620ad2aab37a0b98a81b8', '6680e29817893c388c6e089f', '66814375233606211067dbdb', '6681010dbc2ba2d5202504f5'], ['668621eb69f8b24189420752', '6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8'], ['668621eb69f8b24189420752', '6686213a7e00841dd3b1e343', '667f88dcd004ed06723ba15e', '667f815f75eaed10bd9ad5e6'], ['6680df963d0537f63e29fe42', '668146b09354375e7d0fab11', '6683c66d50c542b909c4f48a', '66861fc42aab37a0b98a81aa', '66861fc42aab37a0b98a81aa', '6680de4aa0b0687efa469963', '6681345e925efa5bba5a51db'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '66814375233606211067dbdb', '6681010dbc2ba2d5202504f5'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e', '6680de4aa0b0687efa469963'], ['668620ad2aab37a0b98a81b8', '6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '6680de4aa0b0687efa469963', '667f88dcd004ed06723ba15e'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e', '6680de4aa0b0687efa469963'], ['668620ad2aab37a0b98a81b8', '6680de4aa0b0687efa469963', '6686213a7e00841dd3b1e343', '667f88dcd004ed06723ba15e'], ['6680de4aa0b0687efa469963', '6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '6680d9f7a0b0687efa469826'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e', '668146b09354375e7d0fab11'], ['6680de4aa0b0687efa469963', '668620ad2aab37a0b98a81b8'], ['6683c66d50c542b909c4f48a', '6681010dbc2ba2d5202504f5', '66810f7474be775d972d8992', '6681034c272da5e070160b76'], ['668620ad2aab37a0b98a81b8', '6680de4aa0b0687efa469963', '6686213a7e00841dd3b1e343'], ['6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['668621eb69f8b24189420752'], ['66861fc42aab37a0b98a81aa', '6680d9f7a0b0687efa469826'], ['668146b09354375e7d0fab11', '6680e29817893c388c6e089f'], ['6680de4aa0b0687efa469963', '667f88dcd004ed06723ba15e', '6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8'], ['668620ad2aab37a0b98a81b8', '6686213a7e00841dd3b1e343', '667f88dcd004ed06723ba15e', '6680de4aa0b0687efa469963'], ['6686213a7e00841dd3b1e343', '6686c651520b31aff4262a78'], ['668620ad2aab37a0b98a81b8', '6686c651520b31aff4262a78', '668620ad2aab37a0b98a81b8', '6686213a7e00841dd3b1e343', '6683c66d50c542b909c4f48a'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '6680de4aa0b0687efa469963'], ['668620ad2aab37a0b98a81b8', '6686213a7e00841dd3b1e343', '6680de4aa0b0687efa469963', '667f88dcd004ed06723ba15e'], ['6680df963d0537f63e29fe42', '6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6680de4aa0b0687efa469963', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e', '6686213a7e00841dd3b1e343', '66861fc42aab37a0b98a81aa'], ['66861fc42aab37a0b98a81aa', '6680d9f7a0b0687efa469826', '668146b09354375e7d0fab11', '668146b09354375e7d0fab11', '6686c651520b31aff4262a78', '6683c66d50c542b909c4f48a', '6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e', '668621eb69f8b24189420752', '6681010dbc2ba2d5202504f5', '6681010dbc2ba2d5202504f5', '6681034c272da5e070160b76', '6680de4aa0b0687efa469963', '667f815f75eaed10bd9ad5e6', '668138fb4e69e24993e01d4a'], ['6681345e925efa5bba5a51db', '66810f7474be775d972d8992', '667f88dcd004ed06723ba15e'], ['6686c651520b31aff4262a78', '668621eb69f8b24189420752', '66861fc42aab37a0b98a81aa', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e', '6686213a7e00841dd3b1e343'], ['6680d9f7a0b0687efa469826', '6681010dbc2ba2d5202504f5'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '668620ad2aab37a0b98a81b8', '6683c66d50c542b909c4f48a'], ['6686213a7e00841dd3b1e343', '667f88dcd004ed06723ba15e', '668620ad2aab37a0b98a81b8'], ['6683c66d50c542b909c4f48a', '668146b09354375e7d0fab11', '6680de4aa0b0687efa469963', '667f815f75eaed10bd9ad5e6', '66861fc42aab37a0b98a81aa', '6686213a7e00841dd3b1e343', '6686c651520b31aff4262a78', '668621eb69f8b24189420752'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826'], ['66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826', '668621eb69f8b24189420752'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826', '668146b09354375e7d0fab11'], ['6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826', '6686213a7e00841dd3b1e343'], ['66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11', '667f815f75eaed10bd9ad5e6', '667f88dcd004ed06723ba15e', '6683c66d50c542b909c4f48a'], ['667f815f75eaed10bd9ad5e6', '667f88dcd004ed06723ba15e', '6680de4aa0b0687efa469963'], ['6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826', '667f815f75eaed10bd9ad5e6', '667f88dcd004ed06723ba15e'], ['66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826', '6686c651520b31aff4262a78'], ['6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826'], ['66861fc42aab37a0b98a81aa', '66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11'], ['66861fc42aab37a0b98a81aa', '6683c66d50c542b909c4f48a', '668146b09354375e7d0fab11', '6680d9f7a0b0687efa469826'], ['6686213a7e00841dd3b1e343', '66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11'], ['66861fc42aab37a0b98a81aa'], ['6686213a7e00841dd3b1e343'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '668146b09354375e7d0fab11'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6681034c272da5e070160b76', '6681345e925efa5bba5a51db', '66810f7474be775d972d8992'], ['6681034c272da5e070160b76', '6681345e925efa5bba5a51db', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '668620ad2aab37a0b98a81b8'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826', '668621eb69f8b24189420752'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '668146b09354375e7d0fab11'], ['6681034c272da5e070160b76', '6681345e925efa5bba5a51db', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826', '668621eb69f8b24189420752'], ['667f815f75eaed10bd9ad5e6', '6681034c272da5e070160b76', '6681345e925efa5bba5a51db', '66810f7474be775d972d8992'], ['66861fc42aab37a0b98a81aa', '6683c66d50c542b909c4f48a', '668146b09354375e7d0fab11', '6680d9f7a0b0687efa469826'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '668146b09354375e7d0fab11'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a'], ['6681034c272da5e070160b76', '6681345e925efa5bba5a51db', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826', '668621eb69f8b24189420752'], ['66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6681034c272da5e070160b76', '6681345e925efa5bba5a51db', '66810f7474be775d972d8992'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826', '668621eb69f8b24189420752'], ['667f815f75eaed10bd9ad5e6'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['668621eb69f8b24189420752', '6680d9f7a0b0687efa469826', '6681449b175ae36ae74b5b27'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a'], ['667f815f75eaed10bd9ad5e6', '6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '6680de4aa0b0687efa469963'], ['6680e29817893c388c6e089f', '668138fb4e69e24993e01d4a', '6681449b175ae36ae74b5b27'], ['6680de4aa0b0687efa469963', '6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f815f75eaed10bd9ad5e6', '667f88dcd004ed06723ba15e'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['667f815f75eaed10bd9ad5e6', '6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '6680de4aa0b0687efa469963'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6686213a7e00841dd3b1e343', '6686c651520b31aff4262a78'], ['66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963', '6683c66d50c542b909c4f48a', '6680d9f7a0b0687efa469826', '668621eb69f8b24189420752'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['668620ad2aab37a0b98a81b8'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['668621eb69f8b24189420752', '6681449b175ae36ae74b5b27'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a', '6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '6681034c272da5e070160b76'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['667f815f75eaed10bd9ad5e6', '6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '6680de4aa0b0687efa469963'], ['66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['6681345e925efa5bba5a51db', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['667f815f75eaed10bd9ad5e6', '6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42'], ['6680decaa0b0687efa4699b1', '6680df963d0537f63e29fe42', '6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a'], ['668621eb69f8b24189420752', '6681449b175ae36ae74b5b27'], ['668621eb69f8b24189420752', '6681449b175ae36ae74b5b27'], ['668621eb69f8b24189420752', '6681449b175ae36ae74b5b27'], ['668621eb69f8b24189420752', '6681449b175ae36ae74b5b27'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '6681034c272da5e070160b76'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '6681034c272da5e070160b76'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '6681034c272da5e070160b76'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '6681034c272da5e070160b76'], ['6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a', '6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681034c272da5e070160b76', '6681345e925efa5bba5a51db', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '6681034c272da5e070160b76'], ['6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a', '6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a', '6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a', '6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '6681034c272da5e070160b76'], ['6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a', '6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a', '6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681345e925efa5bba5a51db', '668138fb4e69e24993e01d4a', '6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8', '667f88dcd004ed06723ba15e'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f', '66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'], ['6681034c272da5e070160b76', '6681345e925efa5bba5a51db', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'], ['66861fc42aab37a0b98a81aa', '668146b09354375e7d0fab11', '667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963']]

# Prepare data for FP-Growth
data_set = defaultdict(int)
for trans in transactions:
    data_set[tuple(trans)] += 1

# Minimum support and confidence
min_support = 1
min_conf = 0.4

# Run FP-Growth
fp_tree, header_table = create_tree(data_set, min_support)
freq_itemsets = []
mine_tree(fp_tree, header_table, min_support, set(), freq_itemsets)

# Sort freq_itemsets by their count in descending order
freq_itemsets = sorted(freq_itemsets, key=lambda x: x[1], reverse=True)

for itemset, count in freq_itemsets:
    print(f"Itemset: {itemset}, Count: {count}")

# Generate association rules
rules = generate_rules(freq_itemsets, min_conf)

# Recommend based on a new transaction
new_transaction = {'6686213a7e00841dd3b1e343','668620ad2aab37a0b98a81b8'}
recommendations = recommend(rules, new_transaction)

# Output recommendations
print(f"Recommendations for the transaction {new_transaction}: {recommendations}")


Itemset: {'667f815f75eaed10bd9ad5e6'}, Count: 72
Itemset: {'6680de4aa0b0687efa469963'}, Count: 69
Itemset: {'6681345e925efa5bba5a51db'}, Count: 63
Itemset: {'667f815f75eaed10bd9ad5e6', '6680de4aa0b0687efa469963'}, Count: 54
Itemset: {'668620ad2aab37a0b98a81b8'}, Count: 51
Itemset: {'6686213a7e00841dd3b1e343'}, Count: 46
Itemset: {'6681010dbc2ba2d5202504f5'}, Count: 40
Itemset: {'668138fb4e69e24993e01d4a'}, Count: 40
Itemset: {'668138fb4e69e24993e01d4a', '6681345e925efa5bba5a51db'}, Count: 38
Itemset: {'6680e29817893c388c6e089f'}, Count: 38
Itemset: {'6686213a7e00841dd3b1e343', '668620ad2aab37a0b98a81b8'}, Count: 37
Itemset: {'6681010dbc2ba2d5202504f5', '6680e29817893c388c6e089f'}, Count: 35
Itemset: {'6683c66d50c542b909c4f48a'}, Count: 30
Itemset: {'667f88dcd004ed06723ba15e'}, Count: 30
Itemset: {'668146b09354375e7d0fab11'}, Count: 30
Itemset: {'66861fc42aab37a0b98a81aa'}, Count: 29
Itemset: {'6680df963d0537f63e29fe42'}, Count: 28
Itemset: {'667f88dcd004ed06723ba15e', '6686213a7e00841d